In [7]:
from anchor import utils
from anchor import anchor_tabular
import pandas as pd
import sklearn.ensemble
import numpy as np
from anchor import anchor_tabular
import os

In [8]:
dataset_folder = '../../datasets/'
dataset = utils.load_dataset('recidivism', balance=True, dataset_folder=dataset_folder, discretize=True)
print(dataset.train[0])

[0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0.]


In [9]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=5)
c.fit(dataset.train, dataset.labels_train)
#print(np.unique(dataset.labels_train))
print('Train', sklearn.metrics.accuracy_score(dataset.labels_train, c.predict(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(dataset.labels_test, c.predict(dataset.test)))

Train 0.935226264418811
Test 0.7531914893617021


In [10]:
transformed_data = []

for r in dataset.train:
    transformed_data.append([dataset.categorical_names[i][int(r_)] for i, r_ in enumerate(r)])        

print(transformed_data[0])

train_data = pd.DataFrame(transformed_data, columns=dataset.feature_names)
print(train_data.shape)
nb_vals = 100

print(dataset.train[0])
print(dataset.train[0].reshape(1, -1))
y=[]
for k in range(0, nb_vals):
    y.append(c.predict(dataset.train[k].reshape(1, -1))[0])


['0', '1', '0', '1', '0', '0', '0', '1', '0', '1', '1', '8.00 < YearsSchool <= 10.00', '0', '21.00 < Age <= 26.00', 'MonthsServed <= 4.00']
(5635, 15)
[0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0.]
[[0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0.]]


In [15]:
import sys
sys.path.append(sys.path[0] + '/..')

import arg_explainer as ae
from importlib import reload
import MinimaliT
reload(ae)
reload(MinimaliT)
import arg_explainer as ae

print(dataset.feature_names)
explainer = ae.ArgTabularExplainer(c, train_data.iloc[0:nb_vals], y, 'recidivism_' + str(nb_vals), compute=True, output_path='../saves')

['Race', 'Alcohol', 'Junky', 'Supervised Release', 'Married', 'Felony', 'WorkRelease', 'Crime against Property', 'Crime against Person', 'Gender', 'Priors', 'YearsSchool', 'PrisonViolations', 'Age', 'MonthsServed']
Generating arguments
1500 potential arg checked ( 0 not minimal)
len  1 : 0 ,  0


KeyboardInterrupt: 

In [6]:
G = explainer.build_attack_graph(compute=True, display_graph=False)
print('edges per node:', np.mean([len(G.edges(n)) for n in G.nodes()]))

explainer.export_graph('asp', '../saves')

1486 1480  args total
recidivism_100
len(R_atk) =  1745735
5 highest degrees: [(frozenset({18, 20}), 1440) (frozenset({23, 7}), 1442)
 (frozenset({23, 15}), 1449) (frozenset({10, 18}), 1454)
 (frozenset({20, 14}), 1463)]
5 lowest degrees: [(frozenset({1, 38, 7, 21, 26}), 835)
 (frozenset({1, 38, 13, 15, 25}), 840)
 (frozenset({1, 38, 7, 11, 25, 29}), 856)
 (frozenset({1, 38, 7, 13, 25}), 864)
 (frozenset({1, 36, 13, 21, 29}), 868)]
edges per node: 1177.1645313553609


In [9]:
#ext_gen = explainer.extension_generator_from_graph()
ext_gen = explainer.extension_generator_from_sat(file='../saves/recidivism_100_sat.txt')
max_covi_ext = explainer.make_selection('max_covi', ext_gen)

Pre-processing file...
Done
Reading ../saves/recidivism_100_sat_preprocessed.txt
Time for selection:  36.6649329662323
Len max_cov_exts:  4


In [13]:
#sel = explainer.make_selection(max_covi_ext, 'max_covc')
res = explainer.apply_inference(max_covi_ext, 'universal')
#print(res)
print(len(max_covi_ext), len(res))
explainer.display_explanations()

4 65
id: 0 coverage: 0 Arg 1/0: None
id: 1 coverage: 4 Arg 1/1: frozenset({'Priors_2', 'Crime against Property_1', 'Race_0', 'YearsSchool_8.00 < YearsSchool <= 10.00'})
id: 2 coverage: 4 Arg 1/4: frozenset({'Priors_1', 'Felony_1', 'Supervised Release_1', 'Race_1', 'MonthsServed_9.00 < MonthsServed <= 24.00'})
id: 3 coverage: 0 Arg 1/0: None
id: 4 coverage: 9 Arg 1/8: frozenset({'Priors_1', 'Race_1', 'Crime against Property_1', 'Felony_0'})
id: 5 coverage: 0 Arg 1/0: None
id: 6 coverage: 4 Arg 1/1: frozenset({'Race_1', 'YearsSchool_YearsSchool > 11.00', 'WorkRelease_0'})
id: 7 coverage: 0 Arg 1/0: None
id: 8 coverage: 9 Arg 1/2: frozenset({'Crime against Person_0', 'Alcohol_1', 'Race_1', 'PrisonViolations_0', 'Priors_2'})
id: 9 coverage: 21 Arg 1/14: frozenset({'YearsSchool_8.00 < YearsSchool <= 10.00', 'Race_1', 'Crime against Property_1', 'Felony_0'})
id: 10 coverage: 0 Arg 1/0: None
id: 11 coverage: 16 Arg 1/4: frozenset({'WorkRelease_0', 'Priors_1', 'Race_1', 'Alcohol_1'})
id: 12 co

In [14]:
res_one = explainer.apply_inference(max_covi_ext, 'one')
print(len(res_one))
explainer.display_explanations()

105
id: 0 coverage: 7 Arg 1/4: frozenset({'Age_21.00 < Age <= 26.00', 'Race_0', 'YearsSchool_8.00 < YearsSchool <= 10.00'})
id: 1 coverage: 9 Arg 1/10: frozenset({'Felony_1', 'Race_0', 'YearsSchool_8.00 < YearsSchool <= 10.00', 'Age_26.00 < Age <= 33.00'})
id: 2 coverage: 4 Arg 1/4: frozenset({'Priors_1', 'Felony_1', 'Supervised Release_1', 'Race_1', 'MonthsServed_9.00 < MonthsServed <= 24.00'})
id: 3 coverage: 3 Arg 1/3: frozenset({'Felony_1', 'Race_0', 'YearsSchool_10.00 < YearsSchool <= 11.00'})
id: 4 coverage: 9 Arg 1/8: frozenset({'Priors_1', 'Race_1', 'Crime against Property_1', 'Felony_0'})
id: 5 coverage: 0 Arg 1/0: None
id: 6 coverage: 4 Arg 1/1: frozenset({'Race_1', 'YearsSchool_YearsSchool > 11.00', 'WorkRelease_0'})
id: 7 coverage: 0 Arg 1/0: None
id: 8 coverage: 9 Arg 1/2: frozenset({'Crime against Person_0', 'Alcohol_1', 'Race_1', 'PrisonViolations_0', 'Priors_2'})
id: 9 coverage: 21 Arg 1/14: frozenset({'YearsSchool_8.00 < YearsSchool <= 10.00', 'Race_1', 'Crime against 

## Sat solver: ASPARTIX

In [ ]:
ext_gen = explainer.extension_generator(file='asp_test.txt')
max_covi_ext = explainer.make_selection(ext_gen, 'max_covi')

In [ ]:
explainer.set_strategy('max_covc', 'existence')
## Define strategy before calling explainer.explain()
explainer.display_explanations()

0.4019384124647283
Number of extensions: 1326480
Time spent on naive extensions: 55.99906277656555 s ( 86.23851943016052 )
len(max_cov_exts)= 1326478 / 1326480
Covc strategy's coverage: 2
Top 5 covs: [2, 2, 2, 2, 2]
id: 0 coverage: 1 Arg 1/12: frozenset({'Relationship_Husband', 'Country_United-States', 'Education_Some-college'})
id: 1 coverage: 1 Arg 1/10: frozenset({'Marital Status_Never-married', 'Age_37.00 < Age <= 48.00'})
id: 2 coverage: 1 Arg 1/22: frozenset({'Age_Age > 48.00', 'Occupation_Exec-managerial', 'Capital Gain_0', 'Workclass_Private'})
id: 3 coverage: 1 Arg 1/18: frozenset({'Hours per week_Hours per week <= 40.00', 'Occupation_Machine-op-inspct'})
id: 4 coverage: 1 Arg 1/17: frozenset({'Capital Loss_0', 'Capital Gain_0', 'Occupation_Sales', 'Hours per week_Hours per week <= 40.00', 'Relationship_Husband'})
id: 5 coverage: 1 Arg 1/8: frozenset({'Occupation_Farming-fishing', 'Hours per week_Hours per week <= 40.00'})
id: 6 coverage: 1 Arg 1/24: frozenset({'Education_Bach

In [ ]:
explainer.set_strategy('max_covc', 'universal')
## Define strategy before calling explainer.explain()
explainer.display_explanations()

0.40616693248272195
Number of extensions: 521076
len(max_cov_exts)= 521074 / 521076
Covc strategy's coverage: 2
Top 5 covs: [2, 2, 2, 2, 2]
id: 0 coverage: 0 Arg 1/0: None
id: 1 coverage: 0 Arg 1/0: None
id: 2 coverage: 0 Arg 1/0: None
id: 3 coverage: 0 Arg 1/0: None
id: 4 coverage: 0 Arg 1/0: None
id: 5 coverage: 0 Arg 1/0: None
id: 6 coverage: 0 Arg 1/0: None
id: 7 coverage: 0 Arg 1/0: None
id: 8 coverage: 0 Arg 1/0: None
id: 9 coverage: 0 Arg 1/0: None
id: 10 coverage: 0 Arg 1/0: None
id: 11 coverage: 0 Arg 1/0: None
id: 12 coverage: 0 Arg 1/0: None
id: 13 coverage: 0 Arg 1/0: None
id: 14 coverage: 0 Arg 1/0: None
id: 15 coverage: 0 Arg 1/0: None
id: 16 coverage: 0 Arg 1/0: None
id: 17 coverage: 0 Arg 1/0: None
id: 18 coverage: 0 Arg 1/0: None
id: 19 coverage: 0 Arg 1/0: None
id: 20 coverage: 0 Arg 1/0: None
id: 21 coverage: 0 Arg 1/0: None
id: 22 coverage: 0 Arg 1/0: None
id: 23 coverage: 0 Arg 1/0: None
id: 24 coverage: 0 Arg 1/0: None
id: 25 coverage: 0 Arg 1/0: None
id: 26 cover

In [ ]:
explainer.set_strategy('max_covi', 'universal')
## Define strategy before calling explainer.explain()
explainer.display_explanations()

0.40616693248272195
Number of extensions: 521076
len(max_cov_exts)= 12 / 521076
Covi strategy's coverage: 60
Top 5 covs: [58, 58, 58, 58, 58]
id: 0 coverage: 0 Arg 1/0: None
id: 1 coverage: 5 Arg 1/4: frozenset({'Relationship_Not-in-family', 'Occupation_Exec-managerial'})
id: 2 coverage: 0 Arg 1/0: None
id: 3 coverage: 3 Arg 1/1: frozenset({'Occupation_Machine-op-inspct', 'Age_28.00 < Age <= 37.00'})
id: 4 coverage: 0 Arg 1/0: None
id: 5 coverage: 2 Arg 1/1: frozenset({'Occupation_Farming-fishing', 'Age_28.00 < Age <= 37.00'})
id: 6 coverage: 2 Arg 1/1: frozenset({'Sex_Female', 'Occupation_Protective-serv'})
id: 7 coverage: 3 Arg 1/1: frozenset({'Occupation_Sales', 'Education_Bachelors'})
id: 8 coverage: 2 Arg 1/1: frozenset({'Occupation_?', 'Sex_Male'})
id: 9 coverage: 3 Arg 1/2: frozenset({'Occupation_Prof-specialty', 'Education_Bachelors', 'Workclass_Private'})
id: 10 coverage: 4 Arg 1/1: frozenset({'Occupation_Protective-serv', 'Sex_Male'})
id: 11 coverage: 1 Arg 1/1: frozenset({'A

In [ ]:
anchor_explainer = anchor_tabular.AnchorTabularExplainer(
    dataset.class_names,
    dataset.feature_names,
    dataset.train,
    dataset.categorical_names)
idx = 0
np.random.seed(1)
instance = dataset.train[idx].reshape(1, -1)[0]
print("Anchor")
print([dataset.categorical_names[i][int(r_)] for i, r_ in enumerate(instance)])

print('Prediction: ', anchor_explainer.class_names[c.predict(dataset.train[0].reshape(1, -1))[0]])
exp = anchor_explainer.explain_instance(dataset.train[idx], c.predict, threshold=0.95)
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

print("Arg:", idx)
print(explainer.explain_instance(idx))

Anchor
['Age > 48.00', 'Self-emp-not-inc', 'Some-college', 'Married-civ-spouse', 'Sales', 'Husband', 'White', 'Male', '2', '0', '40.00 < Hours per week <= 45.00', 'United-States']
Prediction:  b'>50K'
Anchor: Capital Gain = 2
Precision: 1.00
Coverage: 0.07
Arg: 0
id: 0 coverage: 10 Args5/5: {frozenset({'Occupation_Sales', 'Age_Age > 48.00'}), frozenset({'Workclass_Self-emp-not-inc', 'Age_Age > 48.00'}), frozenset({'Workclass_Self-emp-not-inc', 'Occupation_Sales'}), frozenset({'Education_Some-college', 'Workclass_Self-emp-not-inc'}), frozenset({'Education_Some-college', 'Occupation_Sales', 'Country_United-States'})}
None
